In [ ]:
# pip uninstall -y pyarrow
# pip install --no-cache-dir pyarrow==17.0.0
# pip install fastparquet

In [32]:
from datasets import load_dataset
import google.generativeai as genai
import json
from langchain_google_genai import ChatGoogleGenerativeAI
import random
import os


In [42]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = "<API_KEY>"

In [48]:
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [38]:
ds = load_dataset("pubmed_qa", "pqa_labeled")
train_data = ds["train"]

In [64]:
def build_prompt(example, method="zero", shots=[]):
    if method == "zero":
        return f"""You are a biomedical QA assistant.
Answer the following question based only on the context provided.
Provide a detailed descriptive answer with reasoning.

Question: {example['question']}
Context: {example['context']}
Answer:"""
    
    elif method == "one":
        demo = shots[0]
        demo_prompt = f"""Example:
Question: {demo['question']}
Context: {demo['context']}
Answer: {demo['final_decision']} (This is a concise label. 
Now provide detailed reasoning for new examples.)"""
        
        return demo_prompt + f"""

Now answer this:
Question: {example['question']}
Context: {example['context']}
Answer (detailed reasoning):"""
    
    elif method == "few":
        demo_prompt = "\n\n".join(
            [f"Q: {s['question']}\nContext: {s['context']}\nAnswer: {s['final_decision']}" for s in shots]
        )
        return demo_prompt + f"""

Now answer this:
Q: {example['question']}
Context: {example['context']}
Answer (detailed reasoning):"""

In [44]:
def query_gemini(prompt, model="gemini-2.5-flash"):
    response = genai.GenerativeModel(model).generate_content(prompt)
    return response.text.strip()

samples = random.sample(list(train_data), 5)

one_shot_example = random.choice(train_data)
few_shot_examples = random.sample(list(train_data), 3)

In [66]:
# Collect descriptive answers
results = []
for model in ["gemini-2.5-flash", "gemini-2.5-pro"]:
    for s in samples:
        zero_ans = query_gemini(build_prompt(s, "zero"), model)
        one_ans = query_gemini(build_prompt(s, "one", [one_shot_example]), model)
        few_ans = query_gemini(build_prompt(s, "few", few_shot_examples), model)

        results.append({
            "model": model,
            "question": s["question"],
            "context_snippet": s["context"],
            "gold_label": s["final_decision"],  # keep for reference
            "zero_shot_desc": zero_ans,
            "one_shot_desc": one_ans,
            "few_shot_desc": few_ans
        })

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 18.300096436s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 2
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 18
}
]

In [75]:
# Show results
import pandas as pd
df = pd.DataFrame(results)


In [73]:
# Function to display long text neatly
def print_descriptive_results(df, width=100):
    for idx, row in df.iterrows():
        print(f"\n{'='*120}")
        print(f"Model: {row['model']}")
        print(f"Question: {row['question']}")
        print(f"Context Snippet: {row['context_snippet']}")
        print(f"Gold Label: {row['gold_label']}")
        print(f"\n--- Zero-Shot Answer ---\n{row['zero_shot_desc']}")
        print(f"\n--- One-Shot Answer ---\n{row['one_shot_desc']}")
        print(f"\n--- Few-Shot Answer ---\n{row['few_shot_desc']}")
        print(f"{'='*120}\n")

# Display results
print_descriptive_results(df)


Model: gemini-2.5-flash
Question: The promise of specialty pharmaceuticals: are they worth the price?
Context Snippet: {'contexts': ['Specialty pharmaceuticals have evolved beyond their status as niche drugs designed to treat rare conditions and are now poised to become the standard of care in a wide variety of common chronic illnesses. Due in part to the cost of these therapies, payers are increasingly demanding evidence of their value. Determining the value of these medications is hampered by a lack of robust pharmacoeconomic data.', 'To outline emerging strategies and case study examples for the medical and pharmacy benefits management of specialty pharmaceuticals.', 'The promise of specialty pharmaceuticals: increased life expectancy, improved quality of life, enhanced workplace productivity, decreased burden of disease, and reduced health care spending comes at a significant cost. These agents require special handling, administration, patient education, clinical support, and risk